In [14]:
% matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from math import sin, cos, sqrt, atan2, radians

pd.options.display.max_columns = 50

test=pd.read_csv('data/test.csv')
train=pd.read_csv('data/train.csv')
school = pd.read_csv('data/Schools.csv')
subway = pd.read_csv('data/Subways.csv')

train = train.drop_duplicates(train.columns.difference(['key']))
full = pd.concat([train,test],axis=0).reset_index(drop=True)
y=train['transaction_real_price']
full = full.fillna(-999)
full['log_target'] = np.log1p(full['transaction_real_price'])

# School EDA

In [2]:
school.head(20)

,school_code,latitude,longitude,school_class,operation_type,highschool_type,gender,foundation_date,address_by_law
0,S000003511,37.490880,127.015082,elementary,national,NaN,both,1953.1.31,1165010800
1,S000003563,37.577782,127.002915,elementary,national,NaN,both,1946.8.22,1111016800
2,S010000737,37.481366,127.059055,elementary,public,NaN,both,1982.9.20,1168010300
3,S010000738,37.485744,127.058002,elementary,public,NaN,both,1987.11.17,1168010300
4,S010000741,37.480805,127.051891,elementary,public,NaN,both,1983.11.30,1168010300
5,S010000742,37.508423,127.026187,elementary,public,NaN,both,1972.11.23,1168010800
6,S010000743,37.494398,127.065164,elementary,public,NaN,both,1981.9.9,1168010600
7,S010000744,37.492601,127.053554,elementary,public,NaN,both,1979.3.27,1168011800
8,S010000745,37.481135,127.086301,elementary,public,NaN,both,1994.8.5,1168011400
9,S010000746,37.464246,127.105618,elementary,public,NaN,both,1932.11.25,1168011100


In [3]:
uniq = school.nunique().reset_index()
uniq.columns = ['col', 'nunique']
uniq

,col,nunique
0,school_code,1921
1,latitude,1921
2,longitude,1921
3,school_class,3
4,operation_type,3
5,highschool_type,4
6,gender,3
7,foundation_date,1141
8,address_by_law,375


In [4]:
pd.value_counts(school['school_class'])

elementary    901
middle        558
high          462
Name: school_class, dtype: int64

In [5]:
pd.value_counts(school['operation_type'])

public      1444
private      466
national      11
Name: operation_type, dtype: int64

In [6]:
pd.value_counts(school['highschool_type'])

general        269
specialized    104
autonomous      55
objective       34
Name: highschool_type, dtype: int64

In [7]:
pd.value_counts(school['gender'])

both      1524
male       199
female     198
Name: gender, dtype: int64

## Subway EDA

In [8]:
display(subway.head())
print(subway.shape)

,station_id,latitude,longitude,subway_line,address_by_law
0,1,37.555729,126.972145,"1,4,KJ,AP",1.114012e+09
1,2,37.565624,126.976936,"1,2",1.114017e+09
2,3,37.570169,126.983099,1,1.111013e+09
3,4,37.571570,126.991895,"1,3,5",1.111016e+09
4,5,37.570988,127.001921,1,1.111016e+09


(405, 5)


In [9]:
# figure out the kinds of subway_lines
lines = []
for idx, row in subway.iterrows():
    for line in row['subway_line'].split(','):
        lines.append(line)
display(pd.value_counts(lines).index.sort_values())

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', 'AP', 'B1', 'B2', 'B3',
       'B4', 'BD', 'BK', 'DL', 'KC', 'KJ', 'ND', 'US'],
      dtype='object')

## Feature engineering

In [10]:
'''
Feature engineering for school data
1. Label encoding for categorical data :
  - 'school_class', 'operation_type', 'highschool_type', 'gender'
  
2. Feature generation
  - foundation_data => foundation_year
  - combination of 'school_class' and 'operation_type' => class_operation, and then do label encoding
'''

# label encoding for categorical data
school = school.sort_values(by = ['longitude', 'latitude'])
school['school_code'] = pd.factorize(school['school_code'])[0]
school = school.loc[np.arange(0,school.shape[0]),:]

school['school_class'] = pd.factorize(school['school_class'])[0]
school['operation_type'] = pd.factorize(school['operation_type'])[0]
school['highschool_type'] = pd.factorize(school['highschool_type'])[0]
school['gender'] = pd.factorize(school['gender'])[0]

# feature generation for school data
school['foundation_year'] = school['foundation_date'].map(lambda x : x[0:4])
class_operation = []
for idx, row in school.iterrows():
    class_operation.append(str(row['school_class']) + str(row['operation_type']))
school['class_operation'] = pd.factorize(class_operation)[0]
school.head()

,school_code,latitude,longitude,school_class,operation_type,highschool_type,gender,foundation_date,address_by_law,foundation_year,class_operation
0,682,37.490880,127.015082,0,0,-1,0,1953.1.31,1165010800,1953,0
1,624,37.577782,127.002915,0,0,-1,0,1946.8.22,1111016800,1946,0
2,945,37.481366,127.059055,0,1,-1,0,1982.9.20,1168010300,1982,1
3,934,37.485744,127.058002,0,1,-1,0,1987.11.17,1168010300,1987,1
4,890,37.480805,127.051891,0,1,-1,0,1983.11.30,1168010300,1983,1


In [11]:
# Find the nearest subway station and school for each apartment and calculate its distance.
def earth_distance(lat1, lat2, lon1, lon2):
    R=6373.0
    distance=[0]*len(lat2)
    for i in range(len(lat2)):
        d_lon= lon2[i]- lon1
        d_lat= lat2[i]- lat1 
        a = sin(d_lat / 2)**2 + cos(lat1) * cos(lat2[i]) * sin(d_lon / 2)**2 
        c = 2 * atan2(sqrt(a), sqrt(1 - a)) 
        distance[i] = R * c 
    return distance

In [12]:
meta_apartment = full.drop_duplicates(subset = ['apartment_id']).reset_index(drop = True)
meta_apartment = meta_apartment[['apartment_id', 'latitude', 'longitude', 'address_by_law']]
meta_apartment

,apartment_id,latitude,longitude,address_by_law
0,5584,37.585965,127.000231,1111017100
1,5059,37.580511,127.014016,1111017400
2,2816,37.580324,127.011788,1111017400
3,2815,37.575381,126.960804,1111018700
4,9867,37.559200,127.019503,1114016200
5,2818,37.555060,127.014495,1114016200
6,2817,37.549828,127.009284,1114016200
7,2819,37.558170,127.017896,1114016200
8,4059,37.558116,126.965304,1114017100
9,1470,37.537397,127.097622,1121510300


In [18]:
# figure out the kinds of subway_lines
lines = []
for idx, row in subway.iterrows():
    for line in row['subway_line'].split(','):
        lines.append(line)
display(pd.value_counts(lines).index.sort_values())

subway = subway.sort_values(by = ['longitude', 'latitude'])
subway['station_id'] = pd.factorize(subway['station_id'])[0]
subway = subway.loc[np.arange(0,subway.shape[0]),:]

dic = {}
for line in pd.value_counts(lines).index.sort_values():
    dic[line] = [0] * subway.shape[0]

for idx, row in subway.iterrows():
    for line in row['subway_line'].split(','):
        dic[line][idx] = 1 

subway = pd.concat([subway, pd.DataFrame(dic)], axis = 1)
subway.head()

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', 'AP', 'B1', 'B2', 'B3',
       'B4', 'BD', 'BK', 'DL', 'KC', 'KJ', 'ND', 'US'],
      dtype='object')

,station_id,latitude,longitude,subway_line,address_by_law,1,2,3,4,5,6,7,8,9,AP,B1,B2,B3,B4,BD,BK,DL,KC,KJ,ND,US
0,105,37.555729,126.972145,"1,4,KJ,AP",1.114012e+09,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,110,37.565624,126.976936,"1,2",1.114017e+09,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,116,37.570169,126.983099,1,1.111013e+09,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,124,37.571570,126.991895,"1,3,5",1.111016e+09,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,133,37.570988,127.001921,1,1.111016e+09,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
## It takes long time. 
dic = {'min_school' : [], 'min_school_dist' : [],
       'min_subway' : [], 'min_subway_dist' : []}

now = datetime.now()
for apt_id, row in meta_apartment.iterrows():
    sch_dist = earth_distance(row['latitude'], school['latitude'],
                              row['longitude'], school['longitude'])
    sub_dist = earth_distance(row['latitude'], subway['latitude'],
                              row['longitude'], subway['longitude'])
    
    dic['min_school'].append(school.loc[np.argmin(sch_dist), 'school_code']) 
    dic['min_school_dist'].append(np.min(sch_dist))
    
    dic['min_subway'].append(subway.loc[np.argmin(sub_dist), 'station_id']) 
    dic['min_subway_dist'].append(np.min(sub_dist))
    
print(datetime.now() - now)

0:03:49.587679


In [30]:
meta_apartment = pd.concat([meta_apartment, pd.DataFrame(dic)], axis = 1)
meta_apartment.head()

,apartment_id,latitude,longitude,address_by_law,min_school,min_school_dist,min_subway,min_subway_dist
0,5584,37.585965,127.000231,1111017100,S010002342,14.901992,102,24.882336
1,5059,37.580511,127.014016,1111017400,S010002378,6.109126,381,8.067028
2,2816,37.580324,127.011788,1111017400,S010000644,8.018905,381,21.581305
3,2815,37.575381,126.960804,1111018700,S010002376,8.679184,67,19.187394
4,9867,37.559200,127.019503,1114016200,S010002337,7.314980,343,31.254636


In [31]:
meta_apartment = meta_apartment[['apartment_id', 'min_school', 'min_school_dist', 'min_subway', 'min_subway_dist']]
meta_apartment

,apartment_id,min_school,min_school_dist,min_subway,min_subway_dist
0,5584,S010002342,14.901992,102,24.882336
1,5059,S010002378,6.109126,381,8.067028
2,2816,S010000644,8.018905,381,21.581305
3,2815,S010002376,8.679184,67,19.187394
4,9867,S010002337,7.314980,343,31.254636
5,2818,S010002366,17.421649,72,22.380676
6,2817,S010000585,16.374770,380,19.536708
7,2819,S010000633,9.748838,342,28.495923
8,4059,S010000604,11.079480,52,16.746889
9,1470,S010002167,30.147418,23,23.169867


In [32]:
meta_apartment = pd.merge(left = meta_apartment, right = school, left_on = 'min_school', right_on = 'school_code', how = 'left')
meta_apartment = pd.merge(left = meta_apartment, right = subway, left_on = 'min_subway', right_on = 'station_id', how = 'left')

In [35]:
meta_apartment

,apartment_id,min_school,min_school_dist,min_subway,min_subway_dist,school_code,latitude_x,longitude_x,school_class,operation_type,highschool_type,gender,foundation_date,address_by_law_x,foundation_year,class_operation,station_id,latitude_y,longitude_y,subway_line,address_by_law_y,1,2,3,4,5,6,7,8,9,AP,B1,B2,B3,B4,BD,BK,DL,KC,KJ,ND,US
0,5584,S010002342,14.901992,102,24.882336,S010002342,37.585657,127.002564,1,2,-1,1,1907.9.8,1111016900,1907,4,102,37.582418,127.001873,4,1.111017e+09,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5059,S010002378,6.109126,381,8.067028,S010002378,37.581469,127.013964,0,1,-1,0,1971.10.6,1111017400,1971,1,381,37.580002,127.015182,6,1.111017e+09,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2816,S010000644,8.018905,381,21.581305,S010000644,37.579517,127.012760,2,2,2,2,1966.12.8,1111017400,1966,6,381,37.580002,127.015182,6,1.111017e+09,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2815,S010002376,8.679184,67,19.187394,S010002376,37.574137,126.960247,0,1,-1,0,1957.9.28,1111018700,1957,1,67,37.574453,126.957918,3,1.141011e+09,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9867,S010002337,7.314980,343,31.254636,S010002337,37.558278,127.020193,1,1,-1,2,1968.8.6,1114016200,1968,3,343,37.554467,127.020800,5,1.120011e+09,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2818,S010002366,17.421649,72,22.380676,S010002366,37.555760,127.017165,0,2,-1,0,1966.3.10,1114016200,1966,2,72,37.554443,127.011002,"3,6",1.114016e+09,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2817,S010000585,16.374770,380,19.536708,S010000585,37.547641,127.010649,2,1,2,0,1991.10.30,1120011300,1991,7,380,37.547966,127.006821,6,1.114016e+09,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2819,S010000633,9.748838,342,28.495923,S010000633,37.556701,127.017465,2,2,2,0,1956.2.28,1114016200,1956,6,342,37.560205,127.013875,"5,6",NaN,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,4059,S010000604,11.079480,52,16.746889,S010000604,37.557162,126.966773,2,2,2,2,1992.12.29,1114017300,1992,6,52,37.559730,126.963210,"2,5",1.141010e+09,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1470,S010002167,30.147418,23,23.169867,S010002167,37.540815,127.100935,0,1,-1,0,1990.1.20,1121510400,1990,1,23,37.535389,127.094551,2,1.121510e+09,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
